In [5]:
import matplotlib.pyplot as plt
import glob
import numpy as np
from numpy import dot
import cv2
import helpers
from scipy.linalg import inv, block_diag

In [2]:
class Tracker():
    def __init__(self):
        self.id = 0  
        self.box = []
        self.hits = 0 
        self.no_losses = 0
        self.x_state=[]
        self.dt = 1.
        self.F = np.array([[1, self.dt, 0,  0,  0,  0,  0, 0],
                           [0, 1,  0,  0,  0,  0,  0, 0],
                           [0, 0,  1,  self.dt, 0,  0,  0, 0],
                           [0, 0,  0,  1,  0,  0,  0, 0],
                           [0, 0,  0,  0,  1,  self.dt, 0, 0],
                           [0, 0,  0,  0,  0,  1,  0, 0],
                           [0, 0,  0,  0,  0,  0,  1, self.dt],
                           [0, 0,  0,  0,  0,  0,  0,  1]])

        self.H = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 1, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 1, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 1, 0]])


        self.L = 10.0
        self.P = np.diag(self.L*np.ones(8))



        self.Q_comp_mat = np.array([[self.dt**4/4., self.dt**3/2.],
                                    [self.dt**3/2., self.dt**2]])
        self.Q = block_diag(self.Q_comp_mat, self.Q_comp_mat,
                            self.Q_comp_mat, self.Q_comp_mat)


        self.R_scaler = 1.0
        self.R_diag_array = self.R_scaler * np.array([self.L, self.L,
self.L, self.L])
        self.R = np.diag(self.R_diag_array)


    def update_R(self):
        R_diag_array = self.R_scaler * np.array([self.L, self.L,
self.L, self.L])
        self.R = np.diag(R_diag_array)



    def kalman_filter(self, z):
        x = self.x_state
        x = dot(self.F, x)
        self.P = dot(self.F, self.P).dot(self.F.T) + self.Q

        S = dot(self.H, self.P).dot(self.H.T) + self.R
        K = dot(self.P, self.H.T).dot(inv(S))
        y = z - dot(self.H, x) 
        x += dot(K, y)
        self.P = self.P - dot(K, self.H).dot(self.P)
        self.x_state = x.astype(int) 

    def predict_only(self):
        x = self.x_state
        x = dot(self.F, x)
        self.P = dot(self.F, self.P).dot(self.F.T) + self.Q
        self.x_state = x.astype(int)

In [3]:
trk = Tracker()
trk.R_scaler = 1.0/16
trk.update_R()
x_init = np.array([390, 0, 1050, 0, 513, 0, 1278, 0])
x_init_box = [x_init[0], x_init[2], x_init[4], x_init[6]]
z=np.array([399, 1022, 504, 1256])
trk.x_state= x_init.T
trk.kalman_filter(z.T)
x_update =trk.x_state
x_updated_box = [x_update[0], x_update[2], x_update[4], x_update[6]]

print('The initial state is: ', x_init)
print('The measurement is: ', z)
print('The update state is: ', x_update)

img = cv2.VideoCapture('vid.mp4')
while (img.isOpened()):
    ret, frame = img.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Blur
        gray_blurred = cv2.blur(gray, (3, 3))
        plt.figure(figsize=(10, 14))
        helpers.draw_box_label(img, x_init_box, box_color=(0, 255, 0))
        ax = plt.subplot(3, 1, 1)
        plt.imshow(img)
        plt.title('Initial: ' + str(x_init_box))

        helpers.draw_box_label(img, z, box_color=(255, 0, 0))
        ax = plt.subplot(3, 1, 2)
        plt.imshow(img)
        plt.title('Measurement: ' + str(z))

        helpers.draw_box_label(img, x_updated_box)
        ax = plt.subplot(3, 1, 3)
        plt.imshow(img)
        plt.title('Updated: ' + str(x_updated_box))
        plt.show()

NameError: name 'np' is not defined